<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Asignatura Almacenamiento y Captura de Datos</em><br>
    <em>Profesor: Carlos Perez </em><br>

</div>

# **Proyecto: Portal Inmobiliario**

*27 de Diciembre de 2024*

**Nombre Estudiante(s)**: Joaquin Leiva - Victor Saldivia Vera - Cristian Tobar - Constanza Perez

- **Objetivo:** Medir el conocimiento aprendido de base de datos, web scraping y APIs. Para esto se solicita desarrollar
un sencillo producto de consulta de inmuebles con sus comercios cercano.

### **IMPORTACIÓN DE LIBRERÍAS**

In [8]:
import sqlite3
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException


### **VARIABLES**

In [79]:
api_key = ''
op_contrato = 'arriendo'  # opciones: venta, arriendo, arriendo_temporal
op_inmueble = 'departamentos'  # opciones: dpto, casa, oficina
ubicacion_inmueble = 'las condes'
monto_minimo = 500000
monto_maximo = 600000
cant_paginas = 3
radio_busqueda = '300'
busqueda_rubros = ['restaurante', 'supermercado']

### **WEBSCRAPING**

In [80]:
# Inicializar WebDriver 
driver = webdriver.Edge()

# Maximizar ventana
driver.maximize_window()

# Navegar al sitio
url = "https://www.portalinmobiliario.com/"
driver.get(url)
time.sleep(3)

#### **Busqueda de Información**

In [81]:
# Interactuar con los filtros
try:
    # Filtrar por tipo de contrato
    contrato_boton = driver.find_element(By.XPATH, "//button[@aria-label='Tipo de operación']")
    contrato_boton.click()
    
    # Seleccionar el tipo de contrato (Venta, Arriendo, Arriendo Temporal)
    driver.find_element(By.XPATH, f"//span[contains(text(), '{op_contrato.capitalize()}')]").click()

    # Filtrar por tipo de inmueble
    inmueble_boton = driver.find_element(By.XPATH, "//button[@aria-label='Tipo de propiedad']")
    inmueble_boton.click()
    
    # Seleccionar el tipo de inmueble (Departamento, Casa, Oficina)
    driver.find_element(By.XPATH, f"//span[contains(text(), '{op_inmueble.capitalize()}')]").click()

    # Filtrar por comuna y buscar
    comuna_input = driver.find_element(By.XPATH, "//input[@placeholder='Ingresa comuna o ciudad']")
    comuna_input.send_keys(ubicacion_inmueble)
    time.sleep(2)
    comuna_input.send_keys(Keys.ENTER)

    # Clic en botón buscar con espera de tiempo
    boton_buscar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='andes-button__content' and contains(text(), 'Buscar')]"))
    )
    boton_buscar.click()
    
except Exception as e:
    print("Error durante la interacción", e)

#### **Filtro de Precios**

In [82]:
# Se espera a que cargue la página de resultados
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@data-testid='Minimum-price']"))
)

# Filtrar por monto mínimo y máximo
minimo_input = driver.find_element(By.XPATH, "//input[@data-testid='Minimum-price']")
maximo_input = driver.find_element(By.XPATH, "//input[@data-testid='Maximum-price']")

minimo_input.clear()
minimo_input.send_keys(str(monto_minimo))

maximo_input.clear()
maximo_input.send_keys(str(monto_maximo))
maximo_input.send_keys(Keys.ENTER)

#### **Función de Scroll**

In [83]:
# Función para hacer scroll hasta el final de la página
def scroll_down(driver):
    scroll_pause_time = 2  # Tiempo de espera entre scrolls
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Hacer scroll hacia abajo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        # Calcular la nueva altura de la página después de scroll
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

#### **Extracción y Almacenamiento de Resultados**

In [ ]:
# Extraer datos de las páginas de resultados
resultados = []

for pagina in range(cant_paginas):
    # Hacer scroll para cargar todos los anuncios
    scroll_down(driver)
    
    # Recolectar todos los anuncios visibles en la página
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]"))
    )
    anuncios = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]")

    i = 0
    while i < len(anuncios):
        try:
            # Vuelve a recolectar los anuncios después de regresar a la página de resultados
            anuncios = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]//a")

            # Si el índice está fuera de rango, detener el bucle
            if i >= len(anuncios):
                break

            # Hacer clic en el anuncio específico por índice
            driver.execute_script("arguments[0].click();", anuncios[i])
            
            # Esperar a que cargue el contenido del anuncio
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'ui-pdp-title'))
            )
            
            # Extraer información del anuncio
            titulo = driver.find_element(By.XPATH, "//h1[@class='ui-pdp-title']").text
            precio = driver.find_element(By.XPATH, "//span[@class='andes-money-amount__fraction']").text
            moneda = driver.find_element(By.XPATH, "//span[@class='andes-money-amount__currency-symbol']").text
            link = driver.current_url

            # Extracción de dirección completa
            try:
                direccion = driver.find_element(
                    By.XPATH, 
                    "//div[@class='ui-vip-location']//p[@class='ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-media__title']"
                ).text
            except Exception as e:
                direccion = "No disponible"

            # Almacenar resultados sin imprimirlos
            resultados.append([titulo, moneda, precio, direccion, link])

            # Regresar a la página de resultados
            driver.back()
            
            # Scroll de nuevo para asegurar que todos los elementos estén cargados
            scroll_down(driver)

            # Esperar a que la página de resultados esté completamente cargada
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]"))
            )

            i += 1  # Avanzar al siguiente anuncio

        except Exception as e:
            print(f"Error al extraer datos del anuncio: {e}")
            i += 1  # Avanzar al siguiente anuncio incluso si hay error
            continue

    # Si no es la última página, hacer clic en "Siguiente"
    if pagina < cant_paginas - 1:
        try:
            siguiente_boton = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'andes-pagination__button--next')]/a"))
            )
            siguiente_boton.click()

            # Esperar a que cargue la nueva página de resultados
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]"))
            )
        except Exception as e:
            print(f"No se encontró el botón 'Siguiente' o no se pudo hacer clic: {e}")
            break

# DataFrame con resultados
df_resultados = pd.DataFrame(resultados, columns=['Título', 'Moneda', 'Precio', 'Dirección', 'Enlace'])


#### **Impresión del Dataframe**

In [85]:
pd.set_option('display.max_rows', None)
df_resultados


,Título,Moneda,Precio,Dirección,Enlace
0,IMU San Cristóbal,UF,14,"Inés Matte Urrejola 900, Providencia, Las Cond...",https://www.portalinmobiliario.com/arriendo/de...
1,Somma Parque Bustamante - Metro Irarrázaval Ñuñoa,$,518.140,"San Eugenio 401, Parque San Eugenio - Metro Ñu...",https://www.portalinmobiliario.com/arriendo/de...
2,Activa Juan Mitjans,$,550.000,"Juan Mitjans 135, Metro Rodrigo de Araya, Macu...",https://www.portalinmobiliario.com/arriendo/de...
3,INSITU Irarrázaval,$,504.824,"Avenida Irarrázaval 2100 - 2400, Metro Ñuñoa, ...",https://www.portalinmobiliario.com/arriendo/de...
4,Arriendo Depto a pasos del Metro Chile-España ...,$,500.700,"Av. Irarrázaval 2899, Plaza Ñuñoa, Ñuñoa, RM (...",https://www.portalinmobiliario.com/arriendo/de...
5,Nomad Bellet,UF,13,"Antonio Bellet 126, Providencia, Pedro de Vald...",https://www.portalinmobiliario.com/arriendo/de...
6,"Arriendo 2d+2b Carmen 121, Metro Santa Lucía",$,500.000,"Carmen 121, Santiago, Santa Isabel, Santiago, ...",https://www.portalinmobiliario.com/MLC-2808039...
7,Arriendo Departamento Nuevo 2d+2b Avenida Espa...,$,530.000,"Av. España 702, Santiago, Barrio República, Sa...",https://www.portalinmobiliario.com/MLC-1556326...
8,Departamento En Arriendo De 2 Dorm. En Concepción,$,500.000,"Orompello 1470, Concepción, Barrio Poniente, C...",https://www.portalinmobiliario.com/MLC-2814801...
9,Departamento Walker Martínez Id: 143802,$,530.000,"Walker Martínez, La Florida, RM (Metropolitana)",https://www.portalinmobiliario.com/MLC-1552324...
